# Imports

In [ ]:
import sys
import time

import numpy as np
import torch
from bnpm import file_helpers, optimization
import sklearn.utils.class_weight
from torch import nn, optim
from tqdm import tqdm
import sklearn.linear_model
import multiprocessing as mp

import roicat.classification.classifier_util as cu
import scipy.sparse
import roicat
import bnpm.h5_handling
from pathlib import Path
import shutil
import warnings
import umap

import matplotlib.pyplot as plt
import matplotlib.offsetbox
import json
import os
import natsort
from roicat import helpers
# from kymatio.torch import Scattering2D
import gc
import functools

# Specify Initial Parameters

In [3]:
path_params = None # Path(r"")
# directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/00_data_ingestion'
directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/feature_label_combo.npy'
directory_save = r'/Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train'
testing = True
save_ROIs = True
save_latents = True

In [4]:
directory_save = Path(directory_save)
directory_save.mkdir(exist_ok=True, parents=True)

params = file_helpers.json_load(str(Path(path_params).resolve())) if path_params is not None else None

# Data Importing — Suite2p

In [7]:
dir_allOuterFolders = str(Path('/Users/josh/analysis/data/ROICaT/classification/stat_s2p').resolve())

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'
pathSuffixToLabels = 'labels_round2_sesh2.npy'

paths_allStat = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=4,
)
paths_allOps = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToOps,
    depth=4,
)
paths_allLabels = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToLabels,
    depth=4,
)

display(paths_allStat)
display(paths_allOps)

#Import data
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    class_labels=paths_allLabels,
    um_per_pixel=2.0,
    new_or_old_suite2p='new',
    out_height_width=[36, 36],
    type_meanImg='meanImgE',
    verbose=True,
);

['/Users/josh/analysis/data/ROICaT/classification/stat_s2p/stat.npy']

['/Users/josh/analysis/data/ROICaT/classification/stat_s2p/ops.npy']

/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


Starting: Importing FOV images from ops files
Completed: Set FOV_height and FOV_width successfully.
Completed: Imported 1 FOV images.
Completed: Set FOV_images for 1 sessions successfully.
Importing spatial footprints from stat files.


100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


Imported 1 sessions of spatial footprints into sparse arrays.
Completed: Set spatialFootprints for 1 sessions successfully.
Completed: Created session_bool.
Completed: Created centroids.
Staring: Creating centered ROI images from spatial footprints...
Completed: Created ROI images.
Starting: Importing class labels
RH WARNING: class_labels is a list of strings. Assuming each string is a path to a .npy file containing the class labels as a 1D integer array.
Labels and ROI Images match in shapes: Class labels and ROI images have the same number of sessions and the same number of ROIs in each session.
Completed: Imported labels for 1 sessions. Each session has [4898] class labels. Total number of class labels is 4898.


# Data Importing — Hand Labeled Inputs

In [27]:
filepath_labellingInteractive = str(Path('/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/test.ROICaT.labeling.results.pkl').resolve())
labelingInteractive = roicat.helpers.pickle_load(filepath_labellingInteractive)
category_mappings, codes_categories = np.unique(labelingInteractive['labels']['label'], return_inverse=True)

In [28]:
# Import data
data = roicat.data_importing.Data_roicat();
data.set_ROI_images([labelingInteractive['images'][labelingInteractive['labels']['index']]]);
data.set_class_labels([codes_categories]);

RH WARNING: No um_per_pixel provided. We recommend making an educated guess. Assuming 1.0 um per pixel. This will affect the embedding results.
Starting: Importing ROI images
Completed: Imported 1 sessions. Each session has [41] ROIs. Total number of ROIs is 41. The um_per_pixel is 1.0 um per pixel.
Starting: Importing class labels
Labels and ROI Images match in shapes: Class labels and ROI images have the same number of sessions and the same number of ROIs in each session.
Completed: Imported labels for 1 sessions. Each session has [41] class labels. Total number of class labels is 41.


/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


In [2]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
<<<<<<< LOCAL CELL DELETED >>>>>>>
path_params = None # Path(r"")
# directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/00_data_ingestion'
directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/feature_label_combo.npy'
directory_save = r'/Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train'
testing = True
save_ROIs = True
save_latents = True


# path_params = None # Path(r"")
# # filepath_ROIs = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_ROIs.npy'
# filepath_ROIs = None
# filepath_latents = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_latents.npy'
# filepath_labels = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_labels.npy'
# testing = True
# assert (filepath_ROIs is None) != (filepath_latents is None), 'Exactly one of filepath_ROIs or filepath_latents should be set'
# assert Path(filepath_labels).exists(), 'File located at filepath_labels does not exist'

In [3]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
<<<<<<< LOCAL CELL DELETED >>>>>>>
directory_save = Path(directory_save)
directory_save.mkdir(exist_ok=True, parents=True)

labelingRun_interim = {}

if path_params is not None:
    try:
        Path(str((directory_save).resolve())).mkdir(exist_ok=True, parents=True)
        shutil.copy2(path_params, str(Path(directory_save) / Path(path_params).name));
    except Exception as e:
        print(f'JZ: Error copying params to {directory_save}')
        print(e)
tic = time.time()
tictoc = {}
tictoc['start'] = time.time() - tic

params = file_helpers.json_load(str(Path(path_params).resolve())) if path_params is not None else None

<span style="color:red">**<<<<<<< local**</span>

In [ ]:
path_params = None # Path(r"")
# directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/00_data_ingestion'
directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/feature_label_combo.npy'
directory_save = r'/Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train'
testing = True
save_ROIs = True
save_latents = True

In [3]:
dir_allOuterFolders = str(Path('/Users/josh/analysis/data/ROICaT/classification/stat_s2p').resolve())

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'
pathSuffixToLabels = 'labels_round2_sesh2.npy'

paths_allStat = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=4,
)
paths_allOps = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToOps,
    depth=4,
)
paths_allLabels = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToLabels,
    depth=4,
)

display(paths_allStat)
display(paths_allOps)

#Import data
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    class_labels=paths_allLabels,
    um_per_pixel=2.0,
    new_or_old_suite2p='new',
    out_height_width=[36, 36],
    type_meanImg='meanImgE',
    verbose=True,
);

# Data Importing — Hand Labeled Inputs

In [ ]:
filepath_labellingInteractive = str(Path('/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/test.ROICaT.labeling.results.pkl').resolve())
labelingInteractive = roicat.helpers.pickle_load(filepath_labellingInteractive)
category_mappings, codes_categories = np.unique(labelingInteractive['labels']['label'], return_inverse=True)

In [ ]:
# Import data
data = roicat.data_importing.Data_roicat();
data.set_ROI_images([labelingInteractive['images'][labelingInteractive['labels']['index']]]);
data.set_class_labels([codes_categories]);

# Pass Through Network

<span style="color:red">**<<<<<<< local**</span>

<span style="color:red">**<<<<<<< local**</span>

In [5]:
# Neural network embedding distances
roinet = roicat.ROInet.ROInet_embedder(
    device=roicat.util.helpers.set_device('cuda:0'),
    dir_networkFiles=r"/Users/josh/analysis/models",
    download_method="check_local_first",
    download_url="https://osf.io/xwzhp/download",
    download_hash="134b170242141c26b0adbd9e0fd80d0e",
    forward_pass_version="head",
    verbose=True,
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,
    um_per_pixel=data.um_per_pixel,
    pref_plot=False,
    batchSize_dataloader=8,
    pinMemory_dataloader=True,
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=True,
    prefetchFactor_dataloader=2,    
);

# roicat.visualization.display_toggle_image_stack(roinet.ROI_images_rs)

roinet.generate_latents();

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

<span style="color:red">**=======**</span>

In [12]:
path_params = None # Path(r"")
# filepath_ROIs = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_ROIs.npy'
filepath_ROIs = None
filepath_latents = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_latents.npy'
filepath_labels = r'/Users/josh/analysis/outputs/ROICaT/classification/01_labels/arr_labels.npy'
directory_save = '/Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train'
testing = True

assert (filepath_ROIs is None) != (filepath_latents is None), 'Exactly one of filepath_ROIs or filepath_latents should be set'
assert Path(filepath_labels).exists(), 'File located at filepath_labels does not exist'

In [ ]:
filepath_ROIs = Path(filepath_ROIs) if filepath_ROIs else None
filepath_latents = Path(filepath_latents) if filepath_latents else None
filepath_labels = Path(filepath_labels)
directory_save = Path(directory_save)
directory_save.mkdir(exist_ok=True, parents=True)

classifierTrainingRun_interim = {}

if path_params is not None:
    try:
        Path(str((directory_save).resolve())).mkdir(exist_ok=True, parents=True)
        shutil.copy2(path_params, str(Path(directory_save) / Path(path_params).name));
    except Exception as e:
        print(f'JZ: Error copying params to {directory_save}')
        print(e)

In [5]:
if params is None:
    params = {
        "method": "simclr",
        "device": "cuda:0",
        "datatype": "raw_images",
        "hyperparameters_data": {
            "new_or_old_suite2p": "new",
            "out_height_width": [
                36,
                36
            ],
            "type_meanImg": "meanImgE",
            "FOV_images": None,
            "verbose": True,
            "um_per_pixel": 2.0
        },
        "hyperparameters_split": {
            "n_train": 50000,
            "val_size": 0.2,
            "test_size": 0.2,
        },
        "paths": { # TODO: Remove Irrelevant Paths
            "directory_github": "/Users/josh/analysis/github_repos/",
            "directory_data": "/Users/josh/analysis/data/ROICaT/classification/raw_images",
            "filename_rawImages": "sf_concat_rs_sparse.npz",
            "filename_labels": "labels.npy",
            "directory_simclrModel": "/Users/josh/analysis//models",
            "filepath_umapModel": None,
        },
        "hyperparameters_training_simclr": {
            "num_transform_copies": 80,
            "solver": "lbfgs",
            "fit_intercept": True,
            "max_iter": 20000,
            "C": 0.01,
            "tol": 0.001,
            "simclrModel_download_url": "https://osf.io/xwzhp/download",
            "simclrModel_download_hash": "134b170242141c26b0adbd9e0fd80d0e"
        },
        "hyperparameters_augmentations_all": {
            "Scale_image_sum": {
                "sum_all": 1,
                "epsilon": 1e-09,
                "min_sub": True
            },
            "ScaleDynamicRange": {
                "scaler_bounds": [
                    0,
                    1
                ],
                "epsilon": 1e-09
            },
            "WarpPoints": {
                "r": [
                    0.1,
                    0.2
                ],
                "cx": [
                    -0.3,
                    0.3
                ],
                "cy": [
                    -0.3,
                    0.3
                ],
                "dx": [
                    -0.1,
                    0.1
                ],
                "dy": [
                    -0.1,
                    0.1
                ],
                "n_warps": 1,
                "prob": 0.0,
                "img_size_in": [
                    36,
                    36
                ],
                "img_size_out": [
                    224,
                    224
                ]
            },
            "TileChannels": {
                "dim": -3,
                "n_channels": 3
            }
        },
        "run_umap": True,
    }

if not torch.cuda.is_available() and params['device'] != 'cpu':
    warnings.warn('CUDA not available, using CPU')
    params['device'] = torch.device('cpu')

<span style="color:red">**>>>>>>> remote**</span>

<span style="color:red">**=======**</span>

<span style="color:red">**=======**</span>

In [29]:
# Neural network embedding distances
roinet = roicat.ROInet.ROInet_embedder(
    device=roicat.util.helpers.set_device('cuda:0'),
    dir_networkFiles=r"/Users/josh/analysis/models",
    download_method="check_local_first",
    download_url="https://osf.io/xwzhp/download",
    download_hash="134b170242141c26b0adbd9e0fd80d0e",
    forward_pass_version="head",
    verbose=True,
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,
    um_per_pixel=data.um_per_pixel,
    pref_plot=False,
    batchSize_dataloader=8,
    pinMemory_dataloader=True,
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=True,
    prefetchFactor_dataloader=2,    
);

# roicat.visualization.display_toggle_image_stack(roinet.ROI_images_rs)

roinet.generate_latents();

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

devices available: []
no GPU available. Using CPU.
File already exists locally: /Users/josh/analysis/models/ROInet.zip


/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


Hash of local file matches provided hash_hex.
Extracting /Users/josh/analysis/models/ROInet.zip to /Users/josh/analysis/models.
Completed zip extraction.
['/Users/josh/analysis/models/ROInet_classification_20220902', '/Users/josh/analysis/models/ROInet_classification_20220902/ConvNext_tiny__1_0_best__simCLR_wPCA.pth', '/Users/josh/analysis/models/ROInet_classification_20220902/model.py', '/Users/josh/analysis/models/ROInet_classification_20220902/classifier.pkl', '/Users/josh/analysis/models/ROInet_classification_20220902/params.json', '/Users/josh/analysis/models/ROInet_classification_20220902/__pycache__', '/Users/josh/analysis/models/ROInet_classification_20220902/__pycache__/model.cpython-39.pyc']
Imported model from /Users/josh/analysis/models/ROInet_classification_20220902/model.py
Loaded params_model from /Users/josh/analysis/models/ROInet_classification_20220902/params.json


/Users/josh/opt/anaconda3/envs/ROICaT/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/josh/opt/anaconda3/envs/ROICaT/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Generated network using params_model
Loaded state_dict into network from /Users/josh/analysis/models/ROInet_classification_20220902/ConvNext_tiny__1_0_best__simCLR_wPCA.pth
Loaded network onto device cpu
Starting: resizing ROIs
Completed: resizing ROIs
Defined image transformations: Sequential(
  (0): ScaleDynamicRange(scaler_bounds=(0, 1))
  (1): Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
  (2): TileChannels(dim=0)
)
Defined dataset
Defined dataloader
starting: running data through network


100%|██████████| 6/6 [00:13<00:00,  2.24s/it]


completed: running data through network


In [5]:
feature_label_combo = np.load(directory_data, allow_pickle=True)[()]
# TODO: Add alternative data importing method for raw data when feature_label_combo does not exist

<span style="color:red">**>>>>>>> remote**</span>

<span style="color:red">**>>>>>>> remote**</span>

# Train / Validation / Test Split Data, Hyperparameter Tune on Validation Set, and Fit Model

<span style="color:red">**<<<<<<< local**</span>

In [ ]:
INTEGER_MAX = np.iinfo(np.int64(0).dtype).max

# TODO: JZ, IMPLEMENT AS LOOP FOR OPTUNA FOR HYPERPARAMETER TUNING
print('Splitting data...')
# Create data splitting object for stratified sampling into train and test sets (as well as downsampling)
data_splitter = cu.Datasplit(
    features=roinet.latents,
    labels=data.class_labels,
    n_train=INTEGER_MAX,
    val_size=0.2,
    test_size=0.2,
)

print('Calculating class weights...')
num_classes = len(np.unique(data.class_labels))
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(data.class_labels), y=data.class_labels)

print(f'Fitting model to data of dimensions: X: {data_splitter.features_train}, y: {data_splitter.labels_train}...')
# Create lenet model, associated optimizer, loss function, and training tracker
model = sklearn.linear_model.LogisticRegression(
   solver='lbfgs',
   fit_intercept=True,
   max_iter=10000,
   C=1e5,
   class_weight={iClassWeight:classWeight for iClassWeight, classWeight in enumerate(class_weights)},
#    class_weight=class_weights,
)
model.fit(data_splitter.features_train, data_splitter.labels_train)

print(f'Calculating tracker outputs and saving to {directory_save}...')
training_tracker = cu.TrainingTracker(
    directory_save=directory_save,
    class_weights=class_weights, # Class Weights
    n_train_actual=data_splitter.n_train_actual,
    model=({'coef':model.coef_, 'intercept':model.intercept_})
)

y_train_preds = model.predict(data_splitter.features_train).astype(int)
y_train_true = data_splitter.labels_train
y_val_preds = model.predict(data_splitter.features_val).astype(int)
y_val_true = data_splitter.labels_val

# Save training loop results from current epoch for training set
training_tracker.add_accuracy(0, 'accuracy_training', y_train_true, y_train_preds) # Generating training loss
training_tracker.add_confusion_matrix(0, 'confusionMatrix_training', y_train_true, y_train_preds) # Generating confusion matrix

# Save training loop results from current epoch for validation set
training_tracker.add_accuracy(0, 'accuracy_val', y_val_true, y_val_preds) # Generating validation accuracy
training_tracker.add_confusion_matrix(0, 'confusionMatrix_val', y_val_true, y_val_preds) # Generating validation confusion matrix

training_tracker.save_results() # TODO: JZ, ADJUST RESULTS SAVING TO SAVE CONFUSION MATRICES AS NOT A DATAFRAME CSV
training_tracker.print_results()

model_save = {
    'intercept_': model.intercept_,
    'coef_': model.coef_,
    'classes_': model.classes_,
}

Splitting data...
Creating X and y matrices for training data...
Calculating class weights...
Fitting model to data of dimensions: X: (60, 100), y: (60,)...
Calculating tracker outputs and saving to /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train...
Saving results:  /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_training.csv /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_timing.json
self.tictoc={'start': 4.887580871582031e-05, 'loaded_data': 0.6734569072723389, 'splitted_data': 0.6762540340423584, 'completed_training_in_0': 0.7153449058532715}
self.model={'coef': array([[-5.85583578e-03, -2.56761415e-03, -2.85717620e-04,
         1.39980724e-03, -4.69398613e-03,  1.39073784e-03,
        -4.09502665e-03, -6.19066812e-03, -2.53216414e-03,
        -5.05807930e-04, -9.05063450e-04,  1.13785403e-03,
        -7.10335497e-03, -3.48018141e-04, -4.56430364e-03,
        -4.09421531e-03, -9.48537968e-04,  2.061

<span style="color:red">**=======**</span>

In [ ]:
INTEGER_MAX = np.iinfo(np.int64(0).dtype).max

# TODO: JZ, IMPLEMENT AS LOOP FOR OPTUNA FOR HYPERPARAMETER TUNING
print('Splitting data...')
# Create data splitting object for stratified sampling into train and test sets (as well as downsampling)
<<<<<<< local
data_split_val = cu.Datasplit(
<<<<<<< local
    features=features_all,
    labels=labels_all,
    n_train=INTEGER_MAX,
    test_size=params['hyperparameters_split']['val_size'],
=======
    features=feature_label_combo['latents'],
    labels=feature_label_combo['labels'],
=======
data_splitter = cu.Datasplit(
    features=roinet.latents,
    labels=data.class_labels,
>>>>>>> remote
    n_train=INTEGER_MAX,
    val_size=0.2,
    test_size=0.2,
>>>>>>> remote
)
<<<<<<< local
data_split_test = cu.Datasplit(
    features=data_split_val.features_train,
    labels=data_split_val.labels_train,
    n_train=INTEGER_MAX,
<<<<<<< local
    test_size = params['hyperparameters_split']['test_size']/(1 - params['hyperparameters_split']['val_size']),
=======
    test_size = 0.2/(1 - 0.2),
>>>>>>> remote
)

print('Creating X and y matrices for training data...')
X_train = data_split_test.features_train
y_train = data_split_test.labels_train

X_val = data_split_val.features_val
y_val = data_split_val.labels_val

X_test = data_split_val.features_val
y_test = data_split_val.labels_val

y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)
=======
>>>>>>> remote

print('Calculating class weights...')
<<<<<<< local
<<<<<<< local
num_classes = len(np.unique(labels))
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(labels_all), y=labels_all)
=======
num_classes = len(np.unique(feature_label_combo['labels']))
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(feature_label_combo['labels']), y=feature_label_combo['labels'])
>>>>>>> remote
=======
num_classes = len(np.unique(data.class_labels))
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(data.class_labels), y=data.class_labels)
>>>>>>> remote

print(f'Fitting model to data of dimensions: X: {data_splitter.features_train}, y: {data_splitter.labels_train}...')
# Create lenet model, associated optimizer, loss function, and training tracker
model = sklearn.linear_model.LogisticRegression(
<<<<<<< local
   solver=params['hyperparameters_training_simclr']['solver'],
   fit_intercept=params['hyperparameters_training_simclr']['fit_intercept'],
   max_iter=params['hyperparameters_training_simclr']['max_iter'],
   C=params['hyperparameters_training_simclr']['C'],
=======
   solver='lbfgs',
   fit_intercept=True,
   max_iter=10000,
   C=1e5,
>>>>>>> remote
   class_weight={iClassWeight:classWeight for iClassWeight, classWeight in enumerate(class_weights)},
#    class_weight=class_weights,
)
model.fit(data_splitter.features_train, data_splitter.labels_train)

print(f'Calculating tracker outputs and saving to {directory_save}...')
training_tracker = cu.TrainingTracker(
    directory_save=directory_save,
    class_weights=class_weights, # Class Weights
    n_train_actual=data_splitter.n_train_actual,
    model=({'coef':model.coef_, 'intercept':model.intercept_})
)

y_train_preds = model.predict(data_splitter.features_train).astype(int)
y_train_true = data_splitter.labels_train
y_val_preds = model.predict(data_splitter.features_val).astype(int)
y_val_true = data_splitter.labels_val

# Save training loop results from current epoch for training set
training_tracker.add_accuracy(0, 'accuracy_training', y_train_true, y_train_preds) # Generating training loss
training_tracker.add_confusion_matrix(0, 'confusionMatrix_training', y_train_true, y_train_preds) # Generating confusion matrix

# Save training loop results from current epoch for validation set
training_tracker.add_accuracy(0, 'accuracy_val', y_val_true, y_val_preds) # Generating validation accuracy
training_tracker.add_confusion_matrix(0, 'confusionMatrix_val', y_val_true, y_val_preds) # Generating validation confusion matrix

training_tracker.save_results() # TODO: JZ, ADJUST RESULTS SAVING TO SAVE CONFUSION MATRICES AS NOT A DATAFRAME CSV
training_tracker.print_results()

model_save = {
<<<<<<< local
    'intercept': model.intercept_,
    'coefs': model.coef_,
=======
    'intercept_': model.intercept_,
    'coef_': model.coef_,
    'classes_': model.classes_,
>>>>>>> remote
}

<<<<<<< local


<<<<<<< local


Splitting data...
Creating X and y matrices for training data...
Calculating class weights...
Fitting model to data of dimensions: X: (60, 100), y: (60,)...
Calculating tracker outputs and saving to /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train...
Saving results:  /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_training.csv /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_timing.json
self.tictoc={'start': 4.792213439941406e-05, 'loaded_data': 0.7132289409637451, 'splitted_data': 0.7159662246704102, 'completed_training_in_0': 0.7610359191894531}
self.model={'coef': array([[-6.75415296e-03, -5.46746177e-03,  4.79354474e-04,
         1.08285428e-03,  2.15115646e-04, -2.05471810e-03,
        -4.84297000e-04, -5.62958992e-03,  8.15842745e-03,
        -2.81363387e-03,  5.75704830e-04,  4.53531535e-03,
        -9.26651960e-03,  1.37357995e-03, -2.26479577e-03,
        -9.89592445e-04, -1.85910939e-03,  2.919

>>>>>>> remote


<<<<<<< local <unchanged>


Splitting data...
Creating X and y matrices for training data...
Calculating class weights...
Fitting model to data of dimensions: X: torch.Size([2938, 100]), y: (2938,)...
Calculating tracker outputs and saving to /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train...
Saving results:  /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_training.csv /Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train/results_timing.json
self.tictoc={'start': 6.699562072753906e-05, 'loaded_data': 0.2793619632720947, 'splitted_data': 0.28069400787353516, 'completed_training_in_0': 1.374708890914917}
self.model={'coef': array([[ 1.69406483e-04, -5.27381853e-04, -2.60215527e-04,
        -2.68778208e-04,  3.91715543e-04, -4.79283612e-04,
         2.32144896e-04,  2.07393075e-04,  6.25804876e-04,
         3.00665636e-04,  6.02871984e-04,  6.75283610e-04,
        -2.07972300e-04, -1.61987044e-04,  1.55279871e-04,
         4.83258452e-04,  3.8750

>>>>>>> remote <removed>


>>>>>>> remote


<span style="color:red">**>>>>>>> remote**</span>

# Save Outputs

In [ ]:
np.save(str((Path(directory_save) / 'model.npy').resolve()), model_save, allow_pickle=True)
<<<<<<< local
<<<<<<< local
# with open(str((Path(directory_save) / 'classifierTrainingRun_interim.pkl').resolve()), 'wb') as f:
#     np.save(
#         file=f,
#         arr=classifierTrainingRun_interim,
#         allow_pickle=True
#     )
=======

classifierTrainingRun_interim['params_prespecified'] = params
=======
>>>>>>> remote
with open(str((Path(directory_save) / 'classifierTrainingRun_interim.pkl').resolve()), 'wb') as f:
    np.save(
        file=f,
        arr=classifierTrainingRun_interim,
        allow_pickle=True
    )
>>>>>>> remote

print(f'Saved model fit results.')

In [ ]:
<<<<<<< LOCAL CELL DELETED >>>>>>>
np.save(str((Path(directory_save) / 'model.npy').resolve()), model_save, allow_pickle=True)
# with open(str((Path(directory_save) / 'classifierTrainingRun_interim.pkl').resolve()), 'wb') as f:
#     np.save(
#         file=f,
#         arr=classifierTrainingRun_interim,
#         allow_pickle=True
#     )

print(f'Saved model fit results.')